Question-2:
    
Analysis-3: 

To know which SECTION of articles written in 2016 is receiving more COMMENTS in 2016 from which REGION

Community API Data preparation for Analysis:

In [25]:
# Preparing Community API  required for Analysis from the API files downloaded:

#To fetch all .json files 

import os
from glob import glob
import glob

cwd_path = os.getcwd() 
root_dir_path=os.path.abspath(os.path.join(cwd_path, os.pardir)) #To get the parent directory of cwd
root=root_dir_path.replace("\\", "/")
start_dir=root+'/data/NYT/Community_API' #Parent directory

all_files = []
pattern   = "*.json"

#To fetch all .json files present in parent directory and its child directories
for dir,subdir,file in os.walk(start_dir):
    all_files.extend(glob.glob(os.path.join(dir,pattern))) 

#print(all_files[:2])
print('Total number of files in the start_dir and its root directories:',len(all_files))

Total number of files in the start_dir and its root directories: 366


In [26]:
global row #Global variable for storing row_index of excel
row=1 #Starting index

In [27]:
#Function to fetch all the comments of a file and make an entry of required data in excel; 

def read_file_comment(filename):
    import json
    json_file=open(filename)
    #json_file=open('F:/INFO7374-Python/Mid-term_Project/midterm/data/NYT/Archive_API\\2016\\April\\Archive_2016_April.json')
    data=json.load(json_file)
    
    
    # Setting rows and columns indexes
    global row #Telling function that 'row' is a global variable
    col = 0
    
    number_of_articles=len(data[0]['results']['comments'])
    #print('Total number of articles:',number_of_articles)

    for j in range(number_of_articles):
        url=data[0]['results']['comments'][j]['assetURL'] #Url of the article for which comment was made
        commenter=data[0]['results']['comments'][j]['userDisplayName'] 
        commenter_Location=data[0]['results']['comments'][j]['userLocation']
        
        worksheet.write(row, col, url)
        worksheet.write(row, col+1, commenter)
        worksheet.write(row, col+2, commenter_Location)
                
        row+=1
    
        j+=1
        
    

                    

In [28]:
#Writing the data required for analysis to excel sheet

import xlsxwriter
import os

# Create an excel workbook if not present and add a worksheet.
if not os.path.exists('Community_API_Data_For_Analysis.xlsx'):
    workbook = xlsxwriter.Workbook('Community_API_Data_For_Analysis.xlsx')
    
    #Adding worksheet to the workbook
    worksheet = workbook.add_worksheet()

    #Adding headers(title) to the columns
    worksheet.write(0, 0, 'web url of the article') #web_url
    worksheet.write(0, 1, 'Commenter') #commenter
    worksheet.write(0, 2, 'Commenter Location') #commenter location

    #For each file, fetch the articles and make an entry of data required
    for filename in all_files:  
        read_file_comment(filename)
    
    workbook.close()
    print('File has been written.')
    
else:
    print('The file already exists.')

The file already exists.


Analysis-3: To know which section of articles is receiving more comments from which region

In [5]:
#Analysis-3: To know which section of articles is receiving more comments from which region

#Sectionwise-Loctaionwise Comments
import xlrd
import datetime

current_row = 0
sheet_num = 0

section_location_dictionary={} #Dictionary to store section-location-count of comments

# path to the file you want to extract data from
src1 = r'Archive_API_Data_For_Analysis.xlsx' #Archive API file prepared by me for analysis
src2 = r'Community_API_Data_For_Analysis.xlsx' ##Community API file prepared by me for analysis

book1 = xlrd.open_workbook(src1)
book2 = xlrd.open_workbook(src2)

# select the sheet that the data resides in
arc_sheet = book1.sheet_by_index(sheet_num)
comm_sheet = book2.sheet_by_index(sheet_num)

# get the total number of rows
arc_rows = arc_sheet.nrows
print('Total number of archive API rows:',arc_rows-1) # num_rows-1 --> Excluding the header;

comm_rows = comm_sheet.nrows
print('Total number of community API rows:',comm_rows-1) # num_rows-1 --> Excluding the header;

url_dict={}
sec_cnt=0

for j in range(arc_rows):
    if j==0: #to skip the header
         continue
    if not j in url_dict:
        url_dict[arc_sheet.cell_value(j, 0)] = j #Article url,row number in excel sheet

for k in range(comm_rows):
    if k==0: #to skip the header
        continue
        
    asset_url=comm_sheet.cell_value(k, 0) #Asset url
    location=comm_sheet.cell_value(k, 2) #Commenter location
        
    
    if asset_url in url_dict.keys():  # If url of article in comment is present in 2016 url articles 
        row_number=url_dict[asset_url] #Fetch the section of the article
        section=arc_sheet.cell_value(row_number, 2) #Section name
        present=0
    
        if section in section_location_dictionary: #If section is present in dictionary
            length=len(section_location_dictionary[section]) #Fetch the length of the list
            if length==0: #If list is empty append a dictionary
                section_location_dictionary[section].append({location:1})
            else: #If list is not empty
                for i in range(length):
                    if location in section_location_dictionary[section][i]: #Check for a location name in all entries of the list
                        section_location_dictionary[section][i][location]+=1 #If present, increment the number
                        present=1
                if present==0:
                    section_location_dictionary[section].append({location:1}) #If not present, append a dictionary
        else:
            section_location_dictionary[section]=[] #If month is not present in dictionary, add it to dictionary
            length=len(section_location_dictionary[section])
            if length==0:
                section_location_dictionary[section].append({location:1})
    k+=1

print('Length of dictionary i.e. Different Sections of articles:'+str(len(section_location_dictionary)))
        
            

Total number of archive API rows: 61928
Total number of community API rows: 9410
Length of dictionary:6


In [6]:
#Writing Section-wise Location-wise comments written in 2016 for articles of 2016

#Writing the output of the analysis to excel

import xlsxwriter
import os

cwd = os.getcwd() #To access the path of current working directory
path=cwd+'/ana_3'

# Create an excel workbook if not present and add a worksheet.
if not os.path.exists(path+'/'+'Analysis_3_Sectionwise_Loctaionwise_Comments.xlsx'):
    workbook = xlsxwriter.Workbook(path+'/'+'Analysis_3_Sectionwise_Loctaionwise_Comments.xlsx')

    #Adding worksheet to the workbook
    worksheet = workbook.add_worksheet('Section_Loctaionwise_Comments')

    #Adding headers(title) to the columns
    worksheet.write(0, 0, 'Section')
    worksheet.write(0, 1, 'Commenter Location')
    worksheet.write(0, 2, 'Total Number of Articles')

    # Setting rows and columns indexes
    row = 1
    col = 0

    dict_length=len(section_location_dictionary)

    for key in section_location_dictionary.keys():
        worksheet.write(row, col, key)#Section name
    
        location_length=len(section_location_dictionary[key])
        for i in range(location_length):
            for loc_key,value in section_location_dictionary[key][i].items():
                worksheet.write(row, col+1, loc_key) #Location name
                worksheet.write(row, col+2, value) #Count of articles

            row+=1
    workbook.close()
    print('Output Data is written to file.')
    
else:
    print('The file already exists.')



Output Data is written to file.


In [7]:
#Writing the most and least comenting locations section-wise

import xlsxwriter
import os

cwd = os.getcwd() #To access the path of current working directory
path=cwd+'/ana_3'

# Create an excel workbook if not present and add a worksheet.
if not os.path.exists(path+'/'+'Analysis_3_Ouput_Sectionwise_Most_Least_Commenting_Locations.xlsx'):
    workbook = xlsxwriter.Workbook(path+'/'+'Analysis_3_Ouput_Sectionwise_Most_Least_Commenting_Locations.xlsx')
    
    #Adding worksheet to the workbook
    worksheet = workbook.add_worksheet('Most_Least_Comments_Sectionwise')
    
    #Adding headers(title) to the columns
    worksheet.write(0, 0, 'Section')
    worksheet.write(0, 1, 'Location with Most number of comments')
    worksheet.write(0, 2, 'Location with Least number of comments')

    # Setting rows and columns indexes
    row = 1
    col = 0

    for key in section_location_dictionary.keys():
        new_dict={} #Dictionary to store the values of each section
        cnt_list=[]
        location_length=len(section_location_dictionary[key])
        worksheet.write(row,col,key) #Section name
        for i in range(location_length):
            for loc_key,value in section_location_dictionary[key][i].items():
                if loc_key not in new_dict.keys():
                    new_dict[loc_key]=value
                    cnt_list.append(value) #List fetching only the comment counts
                    
        if all(cnt_list[0] == item for item in cnt_list)==False: #If list  contains different entries i.e. 
                                                            #if there are locations with unique comment count
                    
            #Locations with most comments
            #Descending order
            from operator import itemgetter
            desc_sort_dict=sorted(new_dict.items(), key=itemgetter(1),reverse=True) 
            worksheet.write(row,col+1,desc_sort_dict[0][0]+'('+str(desc_sort_dict[0][1])+')') #Top location name(number of comments)
            
            #Locations with least comments
            #Ascending order
            from operator import itemgetter
            asc_sort_dict=sorted(new_dict.items(), key=itemgetter(1),reverse=False) 
            worksheet.write(row,col+2,asc_sort_dict[0][0]+'('+str(asc_sort_dict[0][1])+')') #Bottom location name(number of comments)
    
        else: #If there are locations with same comment count, print both locations as his locations_with_most_comments
            value_to_be_entered="" #Variable that stores all locations with same article count
            #Descending order
            from operator import itemgetter
            desc_sort_dict=sorted(new_dict.items(), key=itemgetter(1),reverse=True) 
            for i in range(len(desc_sort_dict)):
                value_to_be_entered+=desc_sort_dict[i][0]+'('+str(desc_sort_dict[i][1])+')'
                if i<len(desc_sort_dict)-1:
                    value_to_be_entered+=' , '

            worksheet.write(row,col+1,value_to_be_entered) #Top location name(number of comments)
    
            
        row+=1
    
    workbook.close()
    print('Output Data is written to file.')
    
else:
    print('The file already exists.')


Output Data is written to file.
